In [12]:
import os
import fnmatch
import re

def search_string_in_files(directory, search_string):
    matches = []
    pattern = re.compile(search_string)

    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    for line_number, line in enumerate(file, 1):
                        if pattern.search(line):
                            matches.append({
                                'file': filepath,
                                'line_number': line_number,
                                'line': line.strip()
                            })
            except Exception as e:
                print(f"Could not read file {filepath}: {e}")

    return matches

# Example usage
directory_to_search = 'C:/Users/alm/git/Imswitch_descspim/'
#directory_to_search = 'C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/'
print(directory_to_search)

string_to_search = 'power_modulation_mode'

results = search_string_in_files(directory_to_search, string_to_search)

for match in results:
    print(f"Found in {match['file']} on line {match['line_number']}: {match['line']}")


C:/Users/alm/git/Imswitch_descspim/
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\PyCobolt0601LaserManager.py on line 73: def power_modulation_mode(self, enabled):
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\PyCobolt0601LaserManager.py on line 75: self._laser.power_modulation_mode(digital_enabled=True)
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\PyCobolt0601LaserManager.py on line 77: self._laser.power_modulation_mode(digital_enabled=False)
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\__pycache__\PyCobolt0601LaserManager.cpython-310.pyc on line 36:    z!PyCobolt0601LaserManager.setValuec                 C   s   |dkr| j d d S 	 d S )NFr   )r   r0   )r    activer*   r*   r+   setScanModeActive>   s  z*PyCobolt0601LaserManager.setScanModeActivec                 C   s(   |r| j jdd d S | j 

In [1]:
import pycobolt
import time

laser = pycobolt.Cobolt06(port='COM4') # Creates a new Cobolt Laser object.
time.sleep(2)
laser.is_connected()

True

In [6]:
laser.get_mode()

'Syntax error: illegal command'

In [8]:
#laser.modulation_mode()
laser.constant_current(0)

'OK'

In [4]:
laser.constant_power()

'OK'

In [3]:
laser.set_power(5)

'OK'

In [2]:
laser.__repr__()

'Serial number: 12221, Model number: 0488-06-01-0200-100, Wavelength: 488 nm, Type: Cobolt06 Port: COM4'

In [7]:
laser.get_modulation_power()

ValueError: could not convert string to float: 'Syntax error: illegal command'

In [5]:
laser.power_modulation_mode(digital_enabled=True)

In [10]:
laser.digital_modulation(enable=1)

In [2]:
laser2 = pycobolt.Cobolt06DPL(port='COM12') # Creates a new Cobolt Laser object.
time.sleep(2)
laser2.is_connected()

True

In [6]:
def calculate(req_zstep, posConvFac, getPositions):        
        posConvFac = (1 / posConvFac) * 1000           # to µm/du
        print("posConvFac", posConvFac)        # in µm
        req_zstep_du = req_zstep * posConvFac * 1000   # in du
        print("req_zstep", req_zstep)
        print("req_zstep_du", req_zstep_du)

        pos1s, pos1c, pos2s, pos2c  = [float(i.split()[1]) for i in getPositions]
        print("pos1s", pos1s)
        print("pos1c", pos1c)
        print("pos2s", pos2s)
        print("pos2c", pos2c)
        factor = abs(pos1s - pos2s) / abs(pos1c - pos2c)
        print("factor", factor)
        du_camera = [i for i in range(1, 40)]                          # device units camera
        print("du_camera", du_camera)
        dist_camera = [i*posConvFac for i in du_camera]               # distance camera in µm
        print("dist_camera", dist_camera)
        dist_sample = [i*factor for i in dist_camera]                   # distance sample in µm

        print("dist_sample", dist_sample)
        du_sample = [i/posConvFac for i in dist_sample]                 # device units sample
        print("du_sample", du_sample)
        result = [(i, du) for i, du in enumerate(du_sample) if abs(round(du) - du) <= 0.1]
        print("result", result)
        closest_du = min(result, key=lambda x: abs(x[1] - req_zstep_du))
        print("closest_du", closest_du) 
        sample_zstep = closest_du[1] * posConvFac
        print("sample_zstep", sample_zstep)
        camera_zstep = dist_camera[closest_du[0]]
        print("camera_zstep", camera_zstep)

calculate(1, 34554.96, ("Sample: 21165.9050 µm", "Camera: 6222.8114 µm", "Sample: 21365.8763 µm", "Camera: 6288.7065 µm"))

posConvFac 0.028939405515156145
req_zstep 1
req_zstep_du 28.939405515156146
pos1s 21165.905
pos1c 6222.8114
pos2s 21365.8763
pos2c 6288.7065
factor 3.0346915020995358
du_camera [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
dist_camera [0.028939405515156145, 0.05787881103031229, 0.08681821654546844, 0.11575762206062458, 0.14469702757578073, 0.17363643309093688, 0.20257583860609302, 0.23151524412124916, 0.2604546496364053, 0.28939405515156147, 0.3183334606667176, 0.34727286618187375, 0.37621227169702987, 0.40515167721218603, 0.43409108272734215, 0.4630304882424983, 0.4919698937576545, 0.5209092992728106, 0.5498487047879668, 0.5787881103031229, 0.607727515818279, 0.6366669213334352, 0.6656063268485913, 0.6945457323637475, 0.7234851378789037, 0.7524245433940597, 0.7813639489092159, 0.8103033544243721, 0.8392427599395282, 0.8681821654546843, 0.8971215709698405, 0.9260609764849966, 0.9550003

In [ ]:
def calculate(self):
        posConvFac = [(pName, pManager) for pName, pManager in self._master.positionersManager if pManager.forPositioning][0][1]._posConvFac
        posConvFac = (1 / posConvFac) * 1000           # to µm/du
        req_zstep = self._widget.getZstep()            # in µm
        req_zstep_du = req_zstep * posConvFac * 1000   # in du
        
        pos1s, pos1c, pos2s, pos2c  = [float(i.split()[1]) for i in self._widget.getPositions()]
        factor = abs(pos1s - pos2s) / abs(pos1c - pos2c)
        du_camera = [i for i in range(1, 40)]                          # device units camera
        dist_camera = [i*posConvFac for i in du_camera]               # distance camera in µm
        dist_sample = [i*factor for i in dist_camera]                   # distance sample in µm
        du_sample = [i/posConvFac for i in dist_sample]                 # device units sample
        result = [(i, du) for i, du in enumerate(du_sample) if abs(round(du) - du) <= 0.1]
        closest_du = min(result, key=lambda x: abs(x[1] - req_zstep_du))
        sample_zstep = closest_du[1] * posConvFac
        camera_zstep = dist_camera[closest_du[0]]
        self._widget.updateResults(factor, sample_zstep, camera_zstep)